# VCノードの起動、削除

このNotebookではVCP SDKを利用してVCノードを起動、削除する手順を記しています。

## 概要

![構成](images/101-001.png)

### 用語説明

上の図、またはこれ以降に示す図に記されている構成要素についての簡単な説明を以下に記します。

* [VPC](https://aws.amazon.com/jp/vpc/)
  - Amazon が提供している AWS 内の仮想プライベートネットワーク環境
* [Amazon EC2](https://aws.amazon.com/jp/ec2/)
  - Amazon が提供している仮想コンピューティング環境
* [Azure Virtual Network](https://azure.microsoft.com/ja-jp/services/virtual-network/)
  - Microsoft が提供しているクラウド内の仮想プライベートネットワーク環境
* [Azure Virtual Machine](https://azure.microsoft.com/ja-jp/services/virtual-machines/)
  - Microsoft が提供している仮想コンピューティング環境
* VC Controller
  - VCPがユーザに対して提供しているサービス群
* VC Controller Core
  - ユーザからのAPI呼び出しを受けとりUnitGroupの作成、削除などを行う
* Grafana
  - 可視化ツール、ダッシュボードツール
  - VCP では VCノード のモニタリング状況を表示するのに利用している
* VCP SDK
  - VCPの機能を呼び出して VM/BM の作成、削除などを行うPython3のライブラリ
* VCノード
  - Amazon EC2, Azure Virtual Machine などの計算資源を抽象化したVCPのノード
* Unit
  - 同質（同じ計算資源(cpu, memory, ...)、同じクラウド、同じ用途、...）であるVCノードをまとめて扱うためのもの
* UnitGroup
  - 複数のUnitをまとめて扱うためのものです

## 操作手順

大まかな操作手順は以下のようになります。

1. VCP SDKを用いてVCノード(Amazon EC2 インスタンス)を起動する
1. 起動したVCノードに ssh でログインして操作する
1. GrafanaでVCノードの利用状況を確認する
1. Unit内のVCノードをスケールアウト、スケールインする
1. 別のパブリッククラウド(Microsoft Azure)のVCノードを起動する
1. 起動したVCノードをすべて削除する

# VCノード(Amazon EC2インスタンス)の起動

VCP SDKを用いてVCノード(Amazon EC2インスタンス)を起動します。また、起動したVCノードに対してsshでログインして操作を行います。

![EC2の起動](images/101-003.png)

## アクセストークンの入力

VCP SDKを利用するにはVC Controllerのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [1]:
from getpass import getpass
vcc_access_token = getpass()

········


## VCP SDKの初期化

VCP SDKの初期化を行います。

In [2]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(
    vcc_access_token,  # VCCのアクセストークン
)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

この場合はアクセストークンの入力からやり直してください。

`vcp`からは UnitGroup の管理（作成、情報取得）や VCP SDK自体に関する情報取得を行うことができます。例えば、次のセルを実行するとUnitGroupの一覧が表示されます。

> まだUnitGroupを作成していないので、ヘッダー以外はなにも表示されないはずです。

In [3]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate


次のセルを実行すると VCP SDK と VC Controller のバージョンが表示されます。

In [4]:
vcp.version()


vcplib:
  filename: /notebooks/notebook/vcplib/occtr.py
  version: 18.10.3+20181221

vcpsdk:
  filename: /notebooks/notebook/vcpsdk/vcpsdk.py
  version: 18.10.3+20181221

  plugin:
    aws: 1.1+20181031
    aws_disk: 1.0+20181031
    azure: 1.1+20181031
    azure_disk: 1.0+20181031
    sakura: 1.0+20180731
    aic: 1.1+20181031
    abc: 1.1+20181031
    gcp: 1.0+20181031
    onpremises: 1.0+20181031

vc_controller:
  host: 10.0.0.1
  name: vcc1012
  version: 18.10.3+20181221

  plugin:
    azure: 1.1+20181221
    abc: 1.1+20181221
    sakura: 1.0+20181221
    aic: 1.1+20181221
    aws: 1.1+20181221
    gcp: 1.0+20181221

<!---
VcpSDK()のAPIに関する詳細については「[VCPSDK APIドキュメント](/handson/files/DOC/index.html#vcpsdk.vcpsdk.VcpSDK)」を参照してください。
--->

## UnitGroupの作成

`handson101`という名前の UnitGroup を作成します。

In [5]:
# UnitGroupの作成
unit_group = vcp.create_ugroup(
    'handson101'                   # UnitGroupの名前
)

作成したUnitGroupの状態を表示してみます。

> UnitGroup の名前が `handson101` であることと、その状態が `RUNNING` であることなどが確認できるはずです。

In [6]:
print(unit_group)

[Vc]
+ type[compute] name[handson101] vcno(139) state[RUNNING] vcid[57c1a01203e3479f8bca6ffd6d33ef77]



<!---
> UnitGroupがとり得る他の状態に関しては「[VCPSDK APIドキュメント#VC の状態定義](/handson/files/DOC/index.html#id1)」を参照してください。
--->

UnitGroupの一覧を表示してみます。

In [7]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
0,139,compute,handson101,RUNNING,57c1a012...,2019/03/20 05:01:40 UTC


## UnitGroupの状態を確認する

UnitGroupに属している Unit の一覧を表示します。

> まだ、このUnitGroupでは Unit を作成していないのでヘッダー以外は何も表示されません。

In [8]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state


UnitGroupに属しているVCノードの一覧を表示します。

> まだVCノードが存在していないので何も表示されません。

In [9]:
unit_group.df_nodes()

[]

<!---
> `unit_group`に用意されている他のメソッドについては「[VCPSDK APIドキュメント](/handson/files/DOC/index.html#vcpsdk.vcpsdk.VcpUnitClass)」を参照してください。
--->

## VCノードのspecを指定する

Unitを構成するVCノードがどのような設定であるかを指定するためのオブジェクトとしてVCP SDKでは`spec`オブジェクトを用意しています。VCP SDKの利用者は`spec`オブジェクトのプロパティを設定することで Unitに起動するVCノードの設定内容を定義することができます。

`spec`オブジェクトの設定項目の例を以下に示します。

* 仮想マシンのインスタンスタイプ
  - m5.large, c5.large, ...
* 仮想マシンのルートボリュームサイズ(GiB)
* 仮想マシンに割り当てるプライベートIPアドレス
* Unit内に作成するVCノードの数

設定できる項目はクラウドプロバイダ(aws, azure, ...)毎に異なります。

### flavor

`spec`オブジェクトの全てのパラメータを毎回設定するのは煩雑になるので典型的な構成のパラメータセットを事前に定義しています。事前に定義した`spec`パラメータセットのことを VCP SDKでは `flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`の定義もクラウドプロバイダ毎に行っています。

次のセルを実行すると aws用に定義している `flavor` の一覧が表示されます。

In [10]:
vcp.df_flavors('aws')

,flavor,instance_type,volume_size,volume_type
0,small,m4.large,8,standard
1,medium,m4.xlarge,40,standard
2,large,m4.2xlarge,100,standard


### spec オブジェクトを作成する

specオブジェクトを作成します。specオブジェクトを作成するにはプロバイダと`flavor`を指定します。ここでは以下の値を指定します。

* プロバイダ: `aws`
* flavor: `small`

In [11]:
spec = vcp.get_spec('aws', 'small')

作成した`spec`の設定内容を確認してみます。

> `instance_type`パラメータに `flavor` の指定と対応するEC2インスタンスタイプ`m4.large`が設定されていることが確認できるはずです。
> `volume_size`, `volume_type`についてもそれぞれ`flavor`と対応する値が設定されます。

In [12]:
print(spec)


aws
------------------------

    cci_version: "1.0"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.4
    recovery_mode: none
    params:
        {'v': [], 'e': []}
    instance_type: m4.large
    volume_size: 8
    volume_type: standard
    volume_id: []
    cloud_image: default
    tags: {}


`spec`に対しては `flavor`で指定するだけではなく個々のパラメータを直接指定することもできます。例えば、以下のようなものが指定できます。

* num_nodes
  - Unit内に作成するVCノードの数: デフォルト=1
* ip_addresses
  - VCノードに割り当てるプライベートIPアドレスのリスト
  - このパラメータを指定しない場合は利用可能なアドレスが自動的に割り当てられる
* instance_type
  - Amazon EC2のインスタンスタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_size
  - Amazon EC2インスタンスのルートボリュームに割り当てるサイズ(GiB)
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_type
  - Amazon EC2インスタンスのルートボリュームのEBSタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する

試しに `volume_size`を指定してみます。

In [13]:
spec.volume_size = 15

指定した値が`spec`の設定に反映されていることを確認してみます。

In [14]:
print(spec)


aws
------------------------

    cci_version: "1.0"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.4
    recovery_mode: none
    params:
        {'v': [], 'e': []}
    instance_type: m4.large
    volume_size: 15
    volume_type: standard
    volume_id: []
    cloud_image: default
    tags: {}


`volume_size`の値が指定した値に変更されています。

### sshの鍵ファイルを設定する

VCノードにsshでログインするためには事前に公開鍵認証の鍵を登録する必要があります。そのための設定をここで行います。

VCノードに登録する公開鍵認証の**公開鍵**のパスを次のセルで指定してください。

In [15]:
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')

指定した公開鍵を `spec` に設定します。

In [16]:
spec.set_ssh_pubkey(ssh_public_key)

sshの公開鍵に関する設定が`spec`に反映されたことを確認してみます。次のセルを実行すると`params`の`e`の`AUTHORIZED_KEYS`に値が設定されていることが確認できます。

In [17]:
print(spec)


aws
------------------------

    cci_version: "1.0"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.4
    recovery_mode: none
    params:
        {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFEYTh3TlJ0cW5Eck5RbXdod3dWZ21LendwUlYwRnZORzkzU0JjTXJ5WEo2VXpHYTcxTXgxSE5qdnRORlczM3d4bUlLQ0EreUxWSzQyZUdaQWY4QzgzZWRXMkJEdkVhZExDaTR3T0Z6ZzhNbHlNblBFVHBhRmlKMzRSV08xU2xTYTMvZlVxUjJ0SXV1bzRTdmk4SVF5V3V3Z0dTcXlJblpITkNkdDBma3RBMG5FVVRkYVVpdm1iQUxWSWNuNlVCNmpQL0pESDJENmNFNUo0ZjVNckQ1eU1CZzVLZVhreFpwWnRiQlpCK1UyTXlCdVlIeTdGT1AyMWViQ2ovWVZianpJdTFLVVpGZDFkdDBJVjJIT1pqOVJYTzRFUCtpOHJBTlFwcVA4RjBuUE1LNXcwUXNBUEk1RjlSVUFUM051aEZ6SkVCTE9sV1VyRG02WHg4S3puaWpHZnIgYml0X2t1bkBiMTJkNjJhODQ3MWEK']}
    instance_type: m4.large
    volume_size: 15
    volume_type: standard
    volume_id: []
    cloud_image: default
    tags: {}


後でVCノードにログインする際にsshの秘密鍵も必要になるので、ここで設定しておきます。次のセルで**秘密鍵**のパスを指定してください。

In [18]:
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')

公開鍵と秘密鍵が正しいペアであることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [19]:
!grep -q "$(ssh-keygen -y -f {ssh_private_key})" {ssh_public_key}

<!---
他にどのようなパラメータが設定できるのかについては
「[VCPSDK APIドキュメント](/handson/files/DOC/index.html#module-vcpsdk.plugins.spec)」を参照してください。
--->

## Unitの作成とVCノードの起動

Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。

> 処理が完了するまで1分半～2分程度かかります。

In [20]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit(
    'aws-server',               # Unit名の指定
    spec
)

2019-03-20 14:03:20,451 - INFO - BOOTING ... 0 sec
2019-03-20 14:03:25,644 - INFO - BOOTING ... 5 sec
2019-03-20 14:03:30,826 - INFO - BOOTING ... 10 sec
2019-03-20 14:03:36,019 - INFO - BOOTING ... 15 sec
2019-03-20 14:03:41,209 - INFO - BOOTING ... 20 sec
2019-03-20 14:03:46,430 - INFO - BOOTING ... 25 sec
2019-03-20 14:03:51,626 - INFO - BOOTING ... 30 sec
2019-03-20 14:03:56,817 - INFO - BOOTING ... 35 sec
2019-03-20 14:04:02,022 - INFO - BOOTING ... 40 sec
2019-03-20 14:04:07,227 - INFO - BOOTING ... 45 sec
2019-03-20 14:04:12,428 - INFO - BOOTING ... 50 sec
2019-03-20 14:04:17,661 - INFO - BOOTING ... 55 sec
2019-03-20 14:04:22,855 - INFO - BOOTING ... 60 sec
2019-03-20 14:04:28,093 - INFO - BOOTING ... 65 sec
2019-03-20 14:04:33,291 - INFO - BOOTING ... 70 sec
2019-03-20 14:04:38,500 - INFO - BOOTING ... 75 sec
2019-03-20 14:04:43,708 - INFO - unit aws-server is RUNNING


UnitGroupに属しているUnitの一覧表示を行い、Unitが作成されていることを確認します。

In [21]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,139,handson101,RUNNING,57c1a012...,2019/03/20 05:01:40 UTC,aws-server,RUNNING


UnitGroupに属しているVCノードの一覧表示を行い、VCノードが起動していることを確認します。VCノードが正常に起動していることは `node_state` の表示が `RUNNING`になっていることで確認できます。

In [22]:
# VCノードの一覧を DataFrame で表示する
unit_group.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none


<!---
> VCノードがとり得る他の状態に関しては「[VCPSDK APIドキュメント#VCノード の状態定義](/handson/files/DOC/index.html#vcnode)」を参照してください。
--->

ここでは VCノードの起動が完了するまで待ち合わせるモードでUnitの作成を行いましたが、非同期処理でUnit, VCノードの作成を行うこともできます。`create_unit()`の`wait_for`パラメータに`False`を指定すると非同期モードでのUnit, VCノードの作成が行えます。

```
unit = unit_group.create_unit('aws-server', spec, wait_for=False)
```

非同期モードでUnitを作成した場合にUnitやVCノードが起動したことを確認するには、以下に示す方法で状態取得を行ってください。

1. `unit_group.df_units()`
  * Unitの一覧表示の`unit_state`の項目で Unitの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます
 
1. `unit.df_nodes()`, `unit_group.df_nodes()`
  * VCノードの一覧表示の`node_state`の項目で VCノードの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます

非同期モードでUnit作成を行った場合でも、以下のメソッドを利用することで後からVCノード起動の待ち合わせを行うことができます。
* `unit_group.wait_unit_applied(unit_name)`
  - unit_nameで指定されたUnitの状態が `RUNNING` または `ERROR`になるのを待ちます

## 疎通確認

起動した VCノードに対して`ping`を行ってみます。

まずUnitGroup内で起動しているVCノードに割り当てられているプライベートIPアドレスの値を取得して変数 `ip_address`に格納します。

In [23]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses()[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す

print(ip_address)

172.30.2.64


実際に `ping` を行ってみます。

> Codeセルで先頭に `!` をつけるとシェルコマンドが実行できます。また `{}`で囲むことで Python の変数参照やコードの実行に置き換えることができます。詳しくは「[IPython Documentation](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#system-shell-commands)」などを参照してください。

In [24]:
!ping -c 5 {ip_address}

PING 172.30.2.64 (172.30.2.64) 56(84) bytes of data.
64 bytes from 172.30.2.64: icmp_seq=1 ttl=63 time=0.225 ms
64 bytes from 172.30.2.64: icmp_seq=2 ttl=63 time=0.156 ms
64 bytes from 172.30.2.64: icmp_seq=3 ttl=63 time=0.153 ms
64 bytes from 172.30.2.64: icmp_seq=4 ttl=63 time=0.164 ms
64 bytes from 172.30.2.64: icmp_seq=5 ttl=63 time=0.148 ms

--- 172.30.2.64 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 3997ms
rtt min/avg/max/mdev = 0.148/0.169/0.225/0.029 ms


# VCノードに ssh でログインして操作する

起動したVCノードに ssh でログインして操作を行います。

まず、ログインする前に ~/.ssh/known_hosts のホストキーを更新します。

In [25]:
!touch ~/.ssh/known_hosts
# ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keygen -R {ip_address}

# ホストキーの登録
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts

/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4


`ls -la` を実行してみます。

In [26]:
!ssh -i {ssh_private_key} -l root {ip_address} /bin/ls -la

total 20
drwx------    6 root     root          4096 Mar 20 05:04 .
drwxr-xr-x   60 root     root          4096 Mar 20 05:04 ..
drwx------    3 root     root          4096 Jul  6  2017 .cache
drwxr-xr-x    3 root     root          4096 Jul  6  2017 .gem
drwx------    2 root     root          4096 Mar 20 05:04 .ssh


VCノードにsshでログインするにはsshの引数に以下の指定が必要となります。

* sshの identity ファイル(-i)
  - `spec`オブジェクトに設定した公開鍵に対応する秘密鍵
* ユーザ名(-l)
  - `root`
* ログイン先のIPアドレス

sshのオプションを毎回指定するのも煩雑なので変数に設定しておきます。

In [27]:
ssh_opts = f"-i {ssh_private_key} -l root"

VCノードに対して`uname -a`, `df -h` などを実行してみます。

In [28]:
!ssh {ssh_opts} {ip_address} uname -a

Linux ip-172-30-2-64 4.4.0-67-generic #88-Ubuntu SMP Wed Mar 8 16:34:45 UTC 2017 x86_64 Linux


In [29]:
!ssh {ssh_opts} {ip_address} df -h

Filesystem                Size      Used Available Use% Mounted on
none                     14.6G      1.8G     12.1G  13% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                    64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/resolv.conf
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hostname
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hosts
shm                      64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker/aufs


# GrafanaでVCノードの利用状況を確認する

VCPではVCノードの利用状況(CPU負荷、メモリ使用量、ネットワーク）を確認するためGrafanaのダッシュボードを提供しています。

## Grafanaへのログイン

[Grafana](/grafana/d/handson/vcp-metrics?refresh=5s) からダッシュボードを開いてください。

最初にログイン画面が表示されるので ユーザ名、パスワードにそれぞれ `admin`, `admin`を入力してください。 

![Grafanaのログイン画面](../101/media/grafana-login.png)

## 負荷状況のグラフ表示

以下のように表示されます。左半分のグラフがBaseコンテナの負荷状況で、右半分がアプリケーションコンテナの負荷状況のグラフです。

![負荷状況](../101/media/grafana-metrics.png)

VCノードに負荷をかけるために、アプリケーションコンテナで stress コマンドを実行します。

In [30]:
# 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
!ssh {ssh_opts} {ip_address} \
    /usr/local/bin/docker run -td --rm --name stress-0 polinux/stress \
    stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
627beaf3eaaf: Pulling fs layer
4de9f469a6f4: Pulling fs layer
4de9f469a6f4: Verifying Checksum
4de9f469a6f4: Download complete
627beaf3eaaf: Verifying Checksum
627beaf3eaaf: Download complete
627beaf3eaaf: Pull complete
4de9f469a6f4: Pull complete
Digest: sha256:6d1825288ddb6b3cec8d3ac8a488c8ec2449334512ecb938483fc2b25cbbdb9a
Status: Downloaded newer image for polinux/stress:latest
545283aacba97b26750738b782f6f5664128e8aa4f0f543487eda570ece7a365


# VCノードのスケールアウト、スケールイン

![スケールアウト、スケールイン](images/101-004.png)

## VCノードのスケールアウト

Unitには `spec`オブジェクトに指定されている設定と同一の VCノードを作成する機能があります。その機能を利用して Unit にVCノードを追加してみます。

まず、現在のVCノードの状況を確認しておきます。

In [31]:
# VCノードの一覧を DataFrame で表示する
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none


Unit に VCノードを追加するには`unit`オブジェクトに対して `add_nodes()` を呼び出します。次のセルを実行するとUnitに VCノード が追加されます。

> VCノード の起動には 1分半～2分程度かかります。

In [32]:
unit.add_nodes()

2019-03-20 14:04:58,188 - INFO - INDEFINITE ... 0 sec
2019-03-20 14:05:03,402 - INFO - INDEFINITE ... 5 sec
2019-03-20 14:05:08,612 - INFO - INDEFINITE ... 10 sec
2019-03-20 14:05:13,816 - INFO - INDEFINITE ... 15 sec
2019-03-20 14:05:19,018 - INFO - INDEFINITE ... 20 sec
2019-03-20 14:05:24,222 - INFO - INDEFINITE ... 25 sec
2019-03-20 14:05:29,432 - INFO - INDEFINITE ... 30 sec
2019-03-20 14:05:34,629 - INFO - INDEFINITE ... 35 sec
2019-03-20 14:05:39,837 - INFO - INDEFINITE ... 40 sec
2019-03-20 14:05:45,030 - INFO - INDEFINITE ... 45 sec
2019-03-20 14:05:50,239 - INFO - INDEFINITE ... 50 sec
2019-03-20 14:05:55,448 - INFO - INDEFINITE ... 55 sec
2019-03-20 14:06:00,656 - INFO - INDEFINITE ... 60 sec
2019-03-20 14:06:05,876 - INFO - INDEFINITE ... 65 sec
2019-03-20 14:06:11,081 - INFO - INDEFINITE ... 70 sec
2019-03-20 14:06:16,287 - INFO - unit aws-server is RUNNING
2019-03-20 14:06:16,288 - INFO - add_node completed.


`add_node()`を呼び出すことで追加されるVCノードの数はデフォルトでは１ですが、パラメータ`num_add_nodes`を設定することで追加するVCノード数を変更することが出来ます。例えば３ノードを追加するには以下のような指定を行います。

```
unit.add_nodes(num_add_nodes=3)
```
また、追加するVCノードのIPアドレスを指定する場合は、以下のような指定を行います。
```
unit.add_nodes(ip_address='172.30.2.30')
```
他にどんなパラメータを指定することが出来るかについては次のセルのように Codeセルの先頭で `?` を指定することで確認できます。

In [33]:
?unit.add_nodes

追加したVCノードの状態を確認するために、VCノードの一覧表示をおこなってみます。

In [34]:
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none
1,139,handson101,aws-server,RUNNING,2,80e9ed2a...,RUNNING,172.30.2.103,i-0b0823d9f2cecc65e,VCP-1bfc8971-57c1a012,none


起動している全てのVCノードに対して `ping` を行ってみます。

In [35]:
# Unit内のVCノードのIPアドレスのリストを取得する
ip_address = unit.find_ip_addresses()
print(ip_address)

# それぞれのIPアドレスに対して pingコマンドを実行する
for address in ip_address:
    !ping -c 5 {address}
    print()

['172.30.2.64', '172.30.2.103']
PING 172.30.2.64 (172.30.2.64) 56(84) bytes of data.
64 bytes from 172.30.2.64: icmp_seq=1 ttl=63 time=0.173 ms
64 bytes from 172.30.2.64: icmp_seq=2 ttl=63 time=0.168 ms
64 bytes from 172.30.2.64: icmp_seq=3 ttl=63 time=0.162 ms
64 bytes from 172.30.2.64: icmp_seq=4 ttl=63 time=0.170 ms
64 bytes from 172.30.2.64: icmp_seq=5 ttl=63 time=0.164 ms

--- 172.30.2.64 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 3996ms
rtt min/avg/max/mdev = 0.162/0.167/0.173/0.012 ms

PING 172.30.2.103 (172.30.2.103) 56(84) bytes of data.
64 bytes from 172.30.2.103: icmp_seq=1 ttl=63 time=0.282 ms
64 bytes from 172.30.2.103: icmp_seq=2 ttl=63 time=0.193 ms
64 bytes from 172.30.2.103: icmp_seq=3 ttl=63 time=0.212 ms
64 bytes from 172.30.2.103: icmp_seq=4 ttl=63 time=0.214 ms
64 bytes from 172.30.2.103: icmp_seq=5 ttl=63 time=0.209 ms

--- 172.30.2.103 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 3999ms
rtt min/avg/m

VCノードを追加したので ~/.ssh/known_hosts を更新します。

In [36]:
for address in ip_address:
    # ~/.ssh/known_hosts から古いホストキーを削除する
    !ssh-keygen -R {address}
    # ホストキーの登録
    !ssh-keyscan -H {address} >> ~/.ssh/known_hosts

# Host 172.30.2.64 found: line 48 type ED25519
# Host 172.30.2.64 found: line 49 type RSA
# Host 172.30.2.64 found: line 50 type ECDSA
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.103 SSH-2.0-OpenSSH_7.4
# 172.30.2.103 SSH-2.0-OpenSSH_7.4
# 172.30.2.103 SSH-2.0-OpenSSH_7.4


追加で起動した VCノード は同じ Unit に属しているため同じ構成(cpu, memory, disk)になっています。そのことを確認するために、sshでログインして以下のコマンドを実行してみます。

* カーネルバージョン
  - uname -a
* CPU
  - cat /proc/cpuinfo
* メモリ量
  - free
* ディスク容量
  - df -h
  
> ただし aws では同じインスタンスタイプに複数種類の CPU が混在していることがあり CPU が微妙に異なることがあります。

In [37]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} uname -a
    print()

172.30.2.64:
Linux ip-172-30-2-64 4.4.0-67-generic #88-Ubuntu SMP Wed Mar 8 16:34:45 UTC 2017 x86_64 Linux

172.30.2.103:
Linux ip-172-30-2-103 4.4.0-67-generic #88-Ubuntu SMP Wed Mar 8 16:34:45 UTC 2017 x86_64 Linux



In [38]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} cat /proc/cpuinfo | grep 'model name'
    print()

172.30.2.64:
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz

172.30.2.103:
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz



In [39]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} free
    print()

172.30.2.64:
             total       used       free     shared    buffers     cached
Mem:       8173656     810368    7363288      12540      42596     509680
-/+ buffers/cache:     258092    7915564
Swap:            0          0          0

172.30.2.103:
             total       used       free     shared    buffers     cached
Mem:       8173656     742000    7431656      12540      24760     497480
-/+ buffers/cache:     219760    7953896
Swap:            0          0          0



In [40]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} df -h
    print()

172.30.2.64:
Filesystem                Size      Used Available Use% Mounted on
none                     14.6G      1.8G     12.1G  13% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                    64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/resolv.conf
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hostname
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hosts
shm                      64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker/aufs

172.30.2.103:
Filesystem                Size      Used Available Use% Mounted on
none                     14.6G      1.8G     12.1G  13% /
tmpfs                     3.9G         0

先程と同様に、負荷をかけるためにそれぞれのVCノードでアプリケーションコンテナによる stress コマンドの実行を行います。

In [41]:
for idx, address in enumerate(ip_address):
    # 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
    !ssh {ssh_opts} {address} \
        /usr/local/bin/docker run -td --rm --name stress-{idx} polinux/stress \
        stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

6a9ccb6554735030a03ec6be04a59b6662926e00bc73b7327523ea3204ca4272
Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
627beaf3eaaf: Pulling fs layer
4de9f469a6f4: Pulling fs layer
627beaf3eaaf: Download complete
4de9f469a6f4: Verifying Checksum
4de9f469a6f4: Download complete
627beaf3eaaf: Pull complete
4de9f469a6f4: Pull complete
Digest: sha256:6d1825288ddb6b3cec8d3ac8a488c8ec2449334512ecb938483fc2b25cbbdb9a
Status: Downloaded newer image for polinux/stress:latest
9e0f8ea25bd17b5fad87dea9ea1ca82a65cf66c1d5005126d6c68192c5ca7031


[Grafana](/grafana/d/handson/vcp-metrics?refresh=5s)で利用状況を確認してみます。

## VCノードのスケールイン

現在Unitには２つのVCノードがありますが、そのうちの一つを削除してみます。

まず、現在のVCノードの状況を確認しておきます。

In [42]:
# VCノードの一覧を DataFrame で表示する
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none
1,139,handson101,aws-server,RUNNING,2,80e9ed2a...,RUNNING,172.30.2.103,i-0b0823d9f2cecc65e,VCP-1bfc8971-57c1a012,none


Unit から VCノードを削除するにはオブジェクト`unit`に対して `delete_nodes()` を呼び出します。

複数の VCノード のうちどのVCノード を削除するかを引数で指定することができます。ここでは`ip_address`パラメータを指定してVCノードに割り当てられている IPアドレスで VCノードを特定します。

> VCノードを特定するパラメータが指定されなかった場合は削除対象となる VCノードは VCP SDK によって自動的に選択されます。

VCノードを特定するための他のパラメータに指定方法については次のセルを実行すると表示される情報で確認してください。

In [43]:
?unit.delete_nodes

では実際にVCノードを削除します。まず削除対象とするVCノードのIPアドレスを確認します。スケールアウトの節で追加したVCノードのIPアドレスを指定することにします。

In [44]:
# 削除対象とするVCノードの IPアドレスを表示 
print(ip_address[-1])                         # 後から追加した VCノード を削除対象とする

172.30.2.103


次のセルを実行するとUnit からVCノードが削除され、対応するAWS EC2インスタンスも削除されます。

> VCノード の削除には 1分～2分程度かかります。

In [45]:
# VCノードの削除
unit.delete_nodes(
    ip_address=ip_address[-1]     # IPアドレスで削除対象を指定する
)

2019-03-20 14:06:34,930 - INFO - INDEFINITE ... 0 sec
2019-03-20 14:06:40,131 - INFO - INDEFINITE ... 5 sec
2019-03-20 14:06:45,329 - INFO - INDEFINITE ... 10 sec
2019-03-20 14:06:50,536 - INFO - INDEFINITE ... 15 sec
2019-03-20 14:06:55,747 - INFO - INDEFINITE ... 20 sec
2019-03-20 14:07:00,956 - INFO - INDEFINITE ... 25 sec
2019-03-20 14:07:06,172 - INFO - INDEFINITE ... 30 sec
2019-03-20 14:07:11,389 - INFO - INDEFINITE ... 35 sec
2019-03-20 14:07:16,601 - INFO - INDEFINITE ... 40 sec
2019-03-20 14:07:21,809 - INFO - INDEFINITE ... 45 sec
2019-03-20 14:07:27,009 - INFO - INDEFINITE ... 50 sec
2019-03-20 14:07:32,224 - INFO - unit aws-server is RUNNING
2019-03-20 14:07:32,225 - INFO - a node deleted.
2019-03-20 14:07:32,226 - INFO - deleted node(s) completed.


Unit内のVCノードの一覧を確認してみます。VCノードが減っていることが確認できるはずです。

In [46]:
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none


先ほど取得した２つのVCノードのIPアドレスに対して `ping` を行ってみます。
削除されている VCノード に対する `ping` が失敗するので、次のセルは必ずエラーになるはずです。

In [48]:
for address in ip_address:
    !ping -c 5 {address}

PING 172.30.2.64 (172.30.2.64) 56(84) bytes of data.
64 bytes from 172.30.2.64: icmp_seq=1 ttl=63 time=0.177 ms
64 bytes from 172.30.2.64: icmp_seq=2 ttl=63 time=0.138 ms
64 bytes from 172.30.2.64: icmp_seq=3 ttl=63 time=0.165 ms
64 bytes from 172.30.2.64: icmp_seq=4 ttl=63 time=0.161 ms
64 bytes from 172.30.2.64: icmp_seq=5 ttl=63 time=0.168 ms

--- 172.30.2.64 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 3998ms
rtt min/avg/max/mdev = 0.138/0.161/0.177/0.020 ms
PING 172.30.2.103 (172.30.2.103) 56(84) bytes of data.
From 172.30.2.200 icmp_seq=1 Destination Host Unreachable
From 172.30.2.200 icmp_seq=2 Destination Host Unreachable
From 172.30.2.200 icmp_seq=3 Destination Host Unreachable
From 172.30.2.200 icmp_seq=4 Destination Host Unreachable
From 172.30.2.200 icmp_seq=5 Destination Host Unreachable

--- 172.30.2.103 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4024ms
pipe 3


RuntimeError: Unexpected exit code: 1

# 別のクラウド(Microsoft Azure)でVCノードを起動する

![azure](images/101-005.png)

## Unit の追加

先程とは異なる構成のVCノードを追加するので、新たな Unit を作成します。

### VCノードのspecを指定する

awsの場合と同様 `spec` オブジェクトを作成します。

まずはMicrosoft Azure向け `flavor` の定義内容を確認してみます。

In [49]:
vcp.df_flavors('azure')

,flavor,disk_size_gb,managed_disk_type,vm_size
0,small,40,Standard_LRS,Standard_B1s
1,medium,100,Standard_LRS,Standard_D3_v2
2,large,100,Premium_LRS,Standard_D4_v2


では、以下のパラメータを指定して `spec`オブジェクトを作成します。
* プロバイダ: azure
* flavor: medium

In [50]:
spec_azure = vcp.get_spec('azure', 'medium')

作成した`spec_azure`の設定内容を確認してみます。

In [51]:
print(spec_azure)


azure
------------------------

    cci_version: "1.0"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.4
    recovery_mode: none
    params:
        {'v': [], 'e': []}

        vm_size: Standard_D3_v2
        disk_size_gb: 100
        managed_disk_type: Standard_LRS
        ext_managed_disk: []
        cloud_image: default
        ip_address_list: []
    tags: 
        tags: {}


指定した `flavor`に対応する `vm_size`, `disk_size_gb` などが設定されていることが確認できます。

### sshの公開鍵設定

VCノードに登録するsshの公開鍵を `spec_azure`に設定します。先ほど aws 向けの`spec`に設定したのと同じ値を指定しています。

In [52]:
spec_azure.set_ssh_pubkey(ssh_public_key)

### specの設定内容を確認する

`spec_azure`の設定内容を確認してみます。

In [53]:
print(spec_azure)


azure
------------------------

    cci_version: "1.0"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.4
    recovery_mode: none
    params:
        {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFEYTh3TlJ0cW5Eck5RbXdod3dWZ21LendwUlYwRnZORzkzU0JjTXJ5WEo2VXpHYTcxTXgxSE5qdnRORlczM3d4bUlLQ0EreUxWSzQyZUdaQWY4QzgzZWRXMkJEdkVhZExDaTR3T0Z6ZzhNbHlNblBFVHBhRmlKMzRSV08xU2xTYTMvZlVxUjJ0SXV1bzRTdmk4SVF5V3V3Z0dTcXlJblpITkNkdDBma3RBMG5FVVRkYVVpdm1iQUxWSWNuNlVCNmpQL0pESDJENmNFNUo0ZjVNckQ1eU1CZzVLZVhreFpwWnRiQlpCK1UyTXlCdVlIeTdGT1AyMWViQ2ovWVZianpJdTFLVVpGZDFkdDBJVjJIT1pqOVJYTzRFUCtpOHJBTlFwcVA4RjBuUE1LNXcwUXNBUEk1RjlSVUFUM051aEZ6SkVCTE9sV1VyRG02WHg4S3puaWpHZnIgYml0X2t1bkBiMTJkNjJhODQ3MWEK']}

        vm_size: Standard_D3_v2
        disk_size_gb: 100
        managed_disk_type: Standard_LRS
        ext_managed_disk: []
        cloud_image: default
        ip_address_list: []
    tags: 
        tags: {}


aws向けに最初に作成した `spec`オブジェクトと`spec_azure`との差分を確認してみます。

> `spec`オブジェクトの内部では aws は `instance_type`、azure は`vm_size` などそれぞれのクラウドプロバイダに応じたパラメータが設定されていることが確認できます。

In [54]:
from difflib import unified_diff
import sys

sys.stdout.writelines(unified_diff(
    str(spec).splitlines(True),
    str(spec_azure).splitlines(True),
    fromfile='aws',
    tofile='azure',
))

--- aws
+++ azure
@@ -1,10 +1,10 @@
 
-aws
+azure
 ------------------------
 
     cci_version: "1.0"
-    name: aws-server
+    name: 
     description: 
     private_network: default
     num_nodes: 1
@@ -12,10 +12,13 @@
     recovery_mode: none
     params:
         {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFEYTh3TlJ0cW5Eck5RbXdod3dWZ21LendwUlYwRnZORzkzU0JjTXJ5WEo2VXpHYTcxTXgxSE5qdnRORlczM3d4bUlLQ0EreUxWSzQyZUdaQWY4QzgzZWRXMkJEdkVhZExDaTR3T0Z6ZzhNbHlNblBFVHBhRmlKMzRSV08xU2xTYTMvZlVxUjJ0SXV1bzRTdmk4SVF5V3V3Z0dTcXlJblpITkNkdDBma3RBMG5FVVRkYVVpdm1iQUxWSWNuNlVCNmpQL0pESDJENmNFNUo0ZjVNckQ1eU1CZzVLZVhreFpwWnRiQlpCK1UyTXlCdVlIeTdGT1AyMWViQ2ovWVZianpJdTFLVVpGZDFkdDBJVjJIT1pqOVJYTzRFUCtpOHJBTlFwcVA4RjBuUE1LNXcwUXNBUEk1RjlSVUFUM051aEZ6SkVCTE9sV1VyRG02WHg4S3puaWpHZnIgYml0X2t1bkBiMTJkNjJhODQ3MWEK']}
-    instance_type: m4.large
-    volume_size: 15
-    volume_type: standard
-    volume_id: []
-    cloud_image: default
-    tags: {}
+
+        vm_size: S

### Unitの作成とVCノードの起動

まず作成前のUnitGroup状態を確認しておきます。

In [55]:
from IPython.display import display

# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,139,handson101,RUNNING,57c1a012...,2019/03/20 05:01:40 UTC,aws-server,RUNNING


,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none


Unitを作成します。

> 処理が完了するまでには 2～5分程度かかります。

In [56]:
# Unitの作成（同時に VCノードが作成される）
unit2 = unit_group.create_unit(
            'azure-server',    # Unit 名
            spec_azure         # spec オブジェクト
        )

2019-03-20 14:14:39,552 - INFO - BOOTING ... 0 sec
2019-03-20 14:14:44,792 - INFO - BOOTING ... 5 sec
2019-03-20 14:14:50,028 - INFO - BOOTING ... 10 sec
2019-03-20 14:14:55,258 - INFO - BOOTING ... 15 sec
2019-03-20 14:15:00,492 - INFO - BOOTING ... 20 sec
2019-03-20 14:15:05,723 - INFO - BOOTING ... 25 sec
2019-03-20 14:15:10,975 - INFO - BOOTING ... 30 sec
2019-03-20 14:15:16,210 - INFO - BOOTING ... 35 sec
2019-03-20 14:15:21,440 - INFO - BOOTING ... 40 sec
2019-03-20 14:15:26,669 - INFO - BOOTING ... 45 sec
2019-03-20 14:15:31,909 - INFO - BOOTING ... 50 sec
2019-03-20 14:15:37,163 - INFO - BOOTING ... 55 sec
2019-03-20 14:15:42,401 - INFO - BOOTING ... 60 sec
2019-03-20 14:15:47,629 - INFO - BOOTING ... 65 sec
2019-03-20 14:15:52,858 - INFO - BOOTING ... 70 sec
2019-03-20 14:15:58,088 - INFO - BOOTING ... 75 sec
2019-03-20 14:16:03,321 - INFO - BOOTING ... 80 sec
2019-03-20 14:16:08,557 - INFO - BOOTING ... 85 sec
2019-03-20 14:16:13,804 - INFO - BOOTING ... 90 sec
2019-03-20 14:

Unit作成後の状態を確認します。Unit とVCノードがそれぞれ２つあることが確認できるはずです。VCノード一覧ではそれぞれのVCノードの unit_name が異なることが確認できるはずです。またawsとazureのVCノードには**異なるサブネット**のIPアドレスが割り当てられていることが確認できます。

> 各プロバイダにどのようなサブネットを割り当てているのかについては[VCP Manager Web UI](https://vcpdev.ecloud.nii.ac.jp/vcp/v1/ui/occmgr/menu/occtrs) で確認ができます。

In [57]:
# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,139,handson101,RUNNING,57c1a012...,2019/03/20 05:01:40 UTC,aws-server,RUNNING
1,compute,139,handson101,RUNNING,57c1a012...,2019/03/20 05:01:40 UTC,azure-server,RUNNING


,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,139,handson101,aws-server,RUNNING,1,f8c1d249...,RUNNING,172.30.2.64,i-033d73dff33071aa3,VCP-1bfc8971-57c1a012,none
1,139,handson101,azure-server,RUNNING,1,0652e10b...,RUNNING,172.20.2.5,/subscriptions/d47388de-3ed7-4aa6-9e7c-5c09ecf55fd8/resourceGroups/niivcp/providers/Microsoft.Compute/virtualMachines/0652e10bvm,VCP-1bfc8971-57c1a012,none


VCノードを追加したので ~/.ssh/known_hosts を更新します。

In [58]:
for address in unit_group.find_ip_addresses():
    # ~/.ssh/known_hosts から古いホストキーを削除する
    !ssh-keygen -R {address}
    # ホストキーの登録
    !ssh-keyscan -H {address} >> ~/.ssh/known_hosts

# Host 172.30.2.64 found: line 48 type RSA
# Host 172.30.2.64 found: line 49 type ECDSA
# Host 172.30.2.64 found: line 50 type ED25519
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# 172.30.2.64 SSH-2.0-OpenSSH_7.4
# Host 172.20.2.5 found: line 1 type ECDSA
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.20.2.5 SSH-2.0-OpenSSH_7.4
# 172.20.2.5 SSH-2.0-OpenSSH_7.4
# 172.20.2.5 SSH-2.0-OpenSSH_7.4


VCノードに対して`ping` を実行してみます。

In [59]:
# UnitGroup内にあるVCノードのIPアドレスリストを取得する
ip_address = unit_group.find_ip_addresses()

# それぞれのVCノードに対してpingを実行する
for address in ip_address:
    !ping -c 5 {address}
    print()

PING 172.30.2.64 (172.30.2.64) 56(84) bytes of data.
64 bytes from 172.30.2.64: icmp_seq=1 ttl=63 time=0.150 ms
64 bytes from 172.30.2.64: icmp_seq=2 ttl=63 time=0.157 ms
64 bytes from 172.30.2.64: icmp_seq=3 ttl=63 time=0.153 ms
64 bytes from 172.30.2.64: icmp_seq=4 ttl=63 time=0.160 ms
64 bytes from 172.30.2.64: icmp_seq=5 ttl=63 time=0.148 ms

--- 172.30.2.64 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 3996ms
rtt min/avg/max/mdev = 0.148/0.153/0.160/0.014 ms

PING 172.20.2.5 (172.20.2.5) 56(84) bytes of data.
64 bytes from 172.20.2.5: icmp_seq=1 ttl=60 time=7.79 ms
64 bytes from 172.20.2.5: icmp_seq=2 ttl=60 time=8.02 ms
64 bytes from 172.20.2.5: icmp_seq=3 ttl=60 time=7.87 ms
64 bytes from 172.20.2.5: icmp_seq=4 ttl=60 time=8.06 ms
64 bytes from 172.20.2.5: icmp_seq=5 ttl=60 time=8.05 ms

--- 172.20.2.5 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4005ms
rtt min/avg/max/mdev = 7.790/7.961/8.064/0.135 ms



VCPでは複数の異なるクラウドを仮想的に一つの計算基盤（仮想クラウド）のように扱える機能を提供しています。そのため異なるクラウド(aws, azure)で起動したVCノードであっても、追加の設定なしに互いに通信することができます。そのことを確認するために、作成した２つの VCノード の一方から他方に対して `ping` を行ってみます。

In [60]:
# pingを実行するVCノードのIPアドレスを表示する
print(ip_address[0])
print()

!ssh {ssh_opts} {ip_address[0]} \
    ping -c 10 {ip_address[1]}

172.30.2.64

PING 172.20.2.5 (172.20.2.5): 56 data bytes
64 bytes from 172.20.2.5: seq=0 ttl=61 time=7.946 ms
64 bytes from 172.20.2.5: seq=1 ttl=61 time=8.082 ms
64 bytes from 172.20.2.5: seq=2 ttl=61 time=8.614 ms
64 bytes from 172.20.2.5: seq=3 ttl=61 time=8.751 ms
64 bytes from 172.20.2.5: seq=4 ttl=61 time=8.020 ms
64 bytes from 172.20.2.5: seq=5 ttl=61 time=8.808 ms
64 bytes from 172.20.2.5: seq=6 ttl=61 time=9.365 ms
64 bytes from 172.20.2.5: seq=7 ttl=61 time=8.082 ms
64 bytes from 172.20.2.5: seq=8 ttl=61 time=7.560 ms
64 bytes from 172.20.2.5: seq=9 ttl=61 time=8.113 ms

--- 172.20.2.5 ping statistics ---
10 packets transmitted, 10 packets received, 0% packet loss
round-trip min/avg/max = 7.560/8.334/9.365 ms


逆方向で同様のことを行ってみます。

In [61]:
# pingを実行するVCノードのIPアドレスを表示する
print(ip_address[1])
print()

!ssh {ssh_opts} {ip_address[1]} \
    ping -c 10 {ip_address[0]}

172.20.2.5

PING 172.30.2.64 (172.30.2.64): 56 data bytes
64 bytes from 172.30.2.64: seq=0 ttl=61 time=7.815 ms
64 bytes from 172.30.2.64: seq=1 ttl=61 time=8.759 ms
64 bytes from 172.30.2.64: seq=2 ttl=61 time=7.800 ms
64 bytes from 172.30.2.64: seq=3 ttl=61 time=8.000 ms
64 bytes from 172.30.2.64: seq=4 ttl=61 time=8.067 ms
64 bytes from 172.30.2.64: seq=5 ttl=61 time=8.288 ms
64 bytes from 172.30.2.64: seq=6 ttl=61 time=8.009 ms
64 bytes from 172.30.2.64: seq=7 ttl=61 time=8.098 ms
64 bytes from 172.30.2.64: seq=8 ttl=61 time=8.117 ms
64 bytes from 172.30.2.64: seq=9 ttl=61 time=8.347 ms

--- 172.30.2.64 ping statistics ---
10 packets transmitted, 10 packets received, 0% packet loss
round-trip min/avg/max = 7.800/8.130/8.759 ms


## VCノードに ssh でログインして操作する

Azureに追加で起動した VCノード は aws とは異なる構成(cpu, memory, disk)になってるはずです。そのことを確認するために、sshでログインして以下のコマンドを実行してみます。

* カーネルバージョン
  - uname -a
* CPU
  - cat /proc/cpuinfo
* メモリ量
  - free
* ディスク容量
  - df -h

In [62]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} uname -a
    print()

172.30.2.64:
Linux ip-172-30-2-64 4.4.0-67-generic #88-Ubuntu SMP Wed Mar 8 16:34:45 UTC 2017 x86_64 Linux

172.20.2.5:
Linux 0652e10b 4.15.0-1040-azure #44-Ubuntu SMP Thu Feb 21 14:24:01 UTC 2019 x86_64 Linux



In [63]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} cat /proc/cpuinfo | grep 'model name'
    print()

172.30.2.64:
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz

172.20.2.5:
model name	: Intel(R) Xeon(R) CPU E5-2673 v3 @ 2.40GHz
model name	: Intel(R) Xeon(R) CPU E5-2673 v3 @ 2.40GHz
model name	: Intel(R) Xeon(R) CPU E5-2673 v3 @ 2.40GHz
model name	: Intel(R) Xeon(R) CPU E5-2673 v3 @ 2.40GHz



In [64]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} free
    print()

172.30.2.64:
             total       used       free     shared    buffers     cached
Mem:       8173656     801488    7372168      12540      58668     509684
-/+ buffers/cache:     233136    7940520
Swap:            0          0          0

172.20.2.5:
             total       used       free     shared    buffers     cached
Mem:      14362096    1337272   13024824       8940      25888     760628
-/+ buffers/cache:     550756   13811340
Swap:            0          0          0



In [65]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} df -h
    print()

172.30.2.64:
Filesystem                Size      Used Available Use% Mounted on
none                     14.6G      1.8G     12.1G  13% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                    64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/resolv.conf
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hostname
/dev/xvda1               14.6G      1.8G     12.1G  13% /etc/hosts
shm                      64.0M         0     64.0M   0% /dev/shm
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp
/dev/xvda1               14.6G      1.8G     12.1G  13% /var/lib/docker/aufs

172.20.2.5:
Filesystem                Size      Used Available Use% Mounted on
none                     96.9G      1.5G     95.3G   2% /
tmpfs                     6.8G         0  

最後に先程と同様に、負荷をかけるためにそれぞれのVCノードで stress コマンドを実行します。これまで同様 stressコマンドはアプリケーションコンテナとして実行します。

In [66]:
for idx, address in enumerate(ip_address):
    # 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
    !ssh {ssh_opts} {address} \
        /usr/local/bin/docker run -td --rm --name stress-{idx} polinux/stress \
        stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

1a511e2e9b051515f398a0e7998af7584cf2615c61fb9f49b33fa67eed3cdf2f
Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
627beaf3eaaf: Pulling fs layer
4de9f469a6f4: Pulling fs layer
4de9f469a6f4: Download complete
627beaf3eaaf: Download complete
627beaf3eaaf: Pull complete
4de9f469a6f4: Pull complete
Digest: sha256:6d1825288ddb6b3cec8d3ac8a488c8ec2449334512ecb938483fc2b25cbbdb9a
Status: Downloaded newer image for polinux/stress:latest
80b4143b7e27b542fd6a76226f74dd2daf1d5c1d32e5dd08e4aa972368aab04e


[Grafana](/grafana/d/handson/vcp-metrics?refresh=5s)で利用状況を確認してみます。

# 全てのVCノードを削除する

ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

> 全てのリソースの削除には 4～5分程度かかります。

In [67]:
unit_group.cleanup()

2019-03-20 14:20:04,522 - INFO - aws-server is DELETING
azure-server is DELETING ... 0 sec
2019-03-20 14:20:09,759 - INFO - aws-server is DELETING
azure-server is DELETING ... 5 sec
2019-03-20 14:20:15,010 - INFO - aws-server is DELETING
azure-server is DELETING ... 10 sec
2019-03-20 14:20:20,247 - INFO - aws-server is DELETING
azure-server is DELETING ... 15 sec
2019-03-20 14:20:25,486 - INFO - aws-server is DELETING
azure-server is DELETING ... 20 sec
2019-03-20 14:20:30,722 - INFO - aws-server is DELETING
azure-server is DELETING ... 25 sec
2019-03-20 14:20:35,960 - INFO - aws-server is DELETING
azure-server is DELETING ... 30 sec
2019-03-20 14:20:41,196 - INFO - aws-server is DELETING
azure-server is DELETING ... 35 sec
2019-03-20 14:20:46,439 - INFO - aws-server is DELETING
azure-server is DELETING ... 40 sec
2019-03-20 14:20:51,674 - INFO - aws-server is DELETING
azure-server is DELETING ... 45 sec
2019-03-20 14:20:56,873 - INFO - azure-server is DELETING ... 50 sec
2019-03-20 14

削除後の状態を確認してみます。

In [68]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
